# Introduction

This Recommender System Uses the description of a hostel to recommend similar hotels.

Since there is no data available for hostels in Ireland on data libraries so I have scrapped the data from Hostel World website for the experiment.

Let's start by importing the necessary libraries.

In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import re
import random
import cufflinks
pd.options.display.max_columns = 30
from IPython.core.interactiveshell import InteractiveShell
import matplotlib.pyplot as plt
import seaborn as sns

Loading the dataset.

In [ ]:
df_hostels = pd.read_csv("../input/text_based_hostel_new.csv", encoding='latin1')

In [ ]:
df_hostels.head()

# EDA

### Token (vocabulary) Frequency Distribution Before Removing Stop Words

In [ ]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
common_words = get_top_n_words(df_hostels['desc'], 20)
df1 = pd.DataFrame(common_words, columns = ['desc' , 'count'])

df1.head(n=20)

### Token (vocabulary) Frequency Distribution After Removing Stop Words

In [ ]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(df_hostels['desc'], 20)
df2 = pd.DataFrame(common_words, columns = ['desc' , 'count'])

df2.head(n=20)

### Hotel Description Word Count Distribution

In [ ]:
df_hostels['word_count'] = df_hostels['desc'].apply(lambda x: len(str(x).split()))
desc_lengths = list(df_hostels['word_count'])
print("Number of descriptions:",len(desc_lengths),
      "\nAverage word count", np.average(desc_lengths),
      "\nMinimum word count", min(desc_lengths),
      "\nMaximum word count", max(desc_lengths))

Many hostels use description to their full potential, know how to utilize captivating descriptions to appeal to travelers’ emotions to drive direct bookings. Their descriptions may be longer than others.

### Text Preprocessing

The test is pretty clean, we don’t have a lot to do, but just in case.

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
    
df_hostels['desc_clean'] = df_hostels['desc'].apply(clean_text)

### Modeling

* Create a TF-IDF matrix of unigrams, bigrams, and trigrams for each hostel.
* Compute similarity between all hostels using sklearn’s linear_kernel (equivalent to cosine similarity in our case).
* Define a function that takes in hotel name as input and returns the top 10 recommended hostels.

In [ ]:
df_hostels.set_index('name', inplace = True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df_hostels['desc_clean'])
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

indices = pd.Series(df_hostels.index)

def recommendations(name, cosine_similarities = cosine_similarities):
    
    recommended_hotels = []
    
    # gettin the index of the hotel that matches the name
    idx = indices[indices == name].index[0]
    
    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar hotels except itself
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the names of the top 10 matching hotels
    for i in top_10_indexes:
        recommended_hotels.append(list(df_hostels.index)[i])
        
    return recommended_hotels

### Recommendations

Let’s make some recommendations!

In [ ]:
recommendations('Tipperary House Dublin')

Good results, as all the hostels belonged to "Tipperary House Dublin".

Let's test for "Hotel Killarney".

In [ ]:
recommendations('Hotel Killarney')

The results are again awesome.